In [2]:
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI # Changed import
from langchain import agents
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
import os # Import os to check for API key

In [4]:
@tool
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location."""
    # In a real application, this would call a weather API.
    # For demonstration, we'll return a mock response based on current location and given example
    if "Seoul" in location:
        return "It's 25 degrees Celsius and sunny in Seoul." # Current weather for Seoul
    elif "New York" in location:
        return "It's 18 degrees Celsius and cloudy in New York."
    else:
        return f"Weather data not available for {location}."


In [5]:
@tool
def calculate_area_of_rectangle(length: float, width: float) -> float:
    """Calculate the area of a rectangle given its length and width."""
    return length * width

In [6]:
@tool
def search_web(query: str) -> str:
    """Search the web for a given query."""
    # In a real application, this would use a search API like Google Search.
    # For demonstration, we'll return a mock response.
    if "latest AI news" in query:
        return "Google I/O 2024 announcements: Gemini 1.5 Flash, Project Astra, and more."
    else:
        return f"Search results for '{query}': No specific information found."

In [7]:
# Create a list of your tools
tools = [get_current_weather, calculate_area_of_rectangle, search_web]

In [16]:
# 2. Choose an LLM that supports tool calling (CHANGED TO GEMINI)
# Make sure you have your GOOGLE_API_KEY set as an environment variable
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0) # Changed to Gemini


In [17]:
# 3. Create an Agent with these tools

# The prompt remains the same for creating a tool-calling agent.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. You have access to various tools to assist with tasks."), # Simplified system message
    AIMessage(content="I'm here to help! What can I do for you?"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = agents.create_tool_calling_agent(llm, tools, prompt)

In [18]:
# 4. Create an AgentExecutor
agent_executor = agents.AgentExecutor(agent=agent, tools=tools, verbose=True)


In [22]:
# 5. Interact with the Agent

print("--- Agent Interaction 1: Weather ---")
response1 = agent_executor.invoke({"input": "What's the weather like in Seoul?"})
print(response1["output"])

print("\n--- Agent Interaction 2: Calculation ---")
response2 = agent_executor.invoke({"input": "What is the area of a rectangle with length 10.5 and width 5?"})
print(response2["output"])

print("\n--- Agent Interaction 3: Web Search ---")
response3 = agent_executor.invoke({"input": "Can you find out what the latest AI news is?"})
print(response3["output"])

print("\n--- Agent Interaction 4: Multi-step/Combined ---")
response4 = agent_executor.invoke({"input": "What's the weather in New York and what's 12 multiplied by 8?"})
print(response4["output"])

print("\n--- Agent Interaction 5: Complex task ---")
response4 = agent_executor.invoke({"input": "What's 2 times the temperature in New York?"})
print(response4["output"])

--- Agent Interaction 1: Weather ---


> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'location': 'Seoul'}`


It's 25 degrees Celsius and sunny in Seoul.It's 25 degrees Celsius and sunny in Seoul.


> Finished chain.
It's 25 degrees Celsius and sunny in Seoul.


--- Agent Interaction 2: Calculation ---


> Entering new AgentExecutor chain...

Invoking: `calculate_area_of_rectangle` with `{'length': 10.5, 'width': 5.0}`


52.5The area of the rectangle is 52.5.


> Finished chain.
The area of the rectangle is 52.5.


--- Agent Interaction 3: Web Search ---


> Entering new AgentExecutor chain...

Invoking: `search_web` with `{'query': 'latest AI news'}`


Google I/O 2024 announcements: Gemini 1.5 Flash, Project Astra, and more.OK. The latest AI news is about Google I/O 2024 announcements: Gemini 1.5 Flash, Project Astra, and more.


> Finished chain.
OK. The latest AI news is about Google I/O 2024 announcements: Gemini 1.5 Flash, Project Astra, and more.


